In [5]:
import csv
import requests

In [2]:
targets = [
    "BRCA1",
    "BRCA2",
    "TP53",
    "EGFR",
    "HER2",
    "KRAS",
    "ALK",
    "HTT",
    "CFTR",
    "APOE",
    "APOB",
    "PCSK9"
]

In [3]:
target_gmt = {}
with open('data/LINCS_L1000_Chem_Pert_Consensus_Sigs') as o:
    gmt = o.read().strip().split("\n")
    for line in gmt:
        label, _, *genes = line.strip().split("\t")
        direction = " Up" if label.endswith("Up") else " Down"
        label = label.replace(direction, "")
        for gene in genes:
            if gene in targets:
                target = "%s%s"%(gene, direction)
                if target not in target_gmt:
                    target_gmt[target] = []
                target_gmt[target].append(label)

In [4]:
with open("out/LINCS_Biomarkers.dmt", "w") as w:
    csv_writer = csv.writer(w, delimiter="\t")
    for k,v in target_gmt.items():
        csv_writer.writerow([k, '', *v])


In [9]:
payload = {
    "filter": {
        "where": {
            "meta.symbol": {"inq": targets}
        }
    }
}
res = requests.post("https://maayanlab.cloud/sigcom-lincs/metadata-api/entities/find", json=payload)
res.ok

True

In [10]:
len(res.json())

11

In [11]:
entity_map = {}
for i in res.json():
    entity_map[i["id"]] = i["meta"]["symbol"]

In [12]:
entity_map

{'a1bbe12f-6fee-4341-80f0-d8a4a508d959': 'ALK',
 '16fc1452-3480-4573-90a3-8347077636e4': 'APOB',
 '5b763eeb-d308-4897-babe-39bb97368419': 'APOE',
 'd183d2e8-1f7c-446a-a0cc-c1620be9136c': 'BRCA1',
 '58d3a61e-1646-4b80-b477-12df8788c43b': 'BRCA2',
 '88ceba40-1dc3-459e-994c-f42491483cd4': 'CFTR',
 '0ea12571-4972-42d5-a8a2-3d45328b7ea0': 'EGFR',
 'f69db30a-61d0-43a2-9905-10811a4f71df': 'HTT',
 '5811eb5e-970c-48a5-b2ba-31d0598d0824': 'KRAS',
 '141dab86-39d0-41c9-aafd-154faabd5ff6': 'PCSK9',
 '330941f8-ea63-4d2e-8401-bd6dacd05ab9': 'TP53'}

In [16]:
url = 'https://maayanlab.cloud/sigcom-lincs/metadata-api/entities/%s/signatures?filter={"where": {"library": "42cd56da-0ad8-5dad-b27c-fe1d135401b2"}}'
target_dmt = {}

for uid, gene in entity_map.items():
    res = requests.get(url%uid)
    if not res.ok:
        raise("Error")
    else:
        sigs = res.json()
        for sig in sigs:
            direction = sig["direction"]
            drug = sig["meta"]["pert_name"]
            label = "%s %s"%(gene,direction)
            if label not in target_dmt:
                target_dmt[label] = []
            target_dmt[label].append(drug)

In [ ]:
with open("out/LINCS_Biomarkers_v2.dmt", "w") as w:
    csv_writer = csv.writer(w, delimiter="\t")
    for k,v in target_dmt.items():
        csv_writer.writerow([k, '', *v])
